In [ ]:
!pip install -q amplpy ampltools pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.7 MB/s eta 0:00:00


In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_excel("/content/drive/MyDrive/Prescriptive Analytics_Assignment 5/hospital_data.xlsx",
                     sheet_name="Sheet1",
                     index_col=0)

col_names = list(data)
I = [i for i in col_names if "Input" in i]
J = [j for j in col_names if "Output" in j]
Z = list(data.index)

input_data = data[I].stack().to_dict()
output_data = data[J].stack().to_dict()


In [ ]:
ampl.eval ('''

reset;
set I;
set J;
set Z;

param input_data {z in Z, i in I};  # Input data parameter for hospitals and inputs
param output_data {z in Z, j in J};  # Output data parameter for hospitals and outputs
param k symbolic;  # The hospital being checked for efficiency

var x {i in I} >= 0;
var y {j in J} >= 0;

maximize total_value_objective: sum {j in J} output_data[k, j] * y[j];

subject to
  efficiency_constraint {z in Z}: sum {j in J} output_data[z, j] * y[j] <=
                                  sum {i in I} input_data[z, i] * x[i];  # Efficiency constraint for all hospitals
  normalization_constraint: sum {i in I} input_data[k, i] * x[i] == 1;  # Normalization constraint for the chosen hospital

''')


In [ ]:
ampl.set['I'] = I
ampl.set['J'] = J
ampl.set['Z'] = Z

ampl.param['input_data'] = input_data
ampl.param['output_data'] = output_data
ampl.param['k'] = 1


In [ ]:
ampl.eval('''expand;''')

maximize total_value_objective:
	51.1574*y['Output 1'] + 32.7153*y['Output 2'] + 66.2448*y['Output 3']
	 + 35.7387*y['Output 4'] + 49.023*y['Output 5'] + 49.2987*y['Output 6']
	 + 39.5518*y['Output 7'];

subject to efficiency_constraint[1]:
	-83.7033*x['Input 1'] - 2.994*x['Input 2'] - 8.048*x['Input 3'] + 
	51.1574*y['Output 1'] + 32.7153*y['Output 2'] + 66.2448*y['Output 3']
	 + 35.7387*y['Output 4'] + 49.023*y['Output 5'] + 49.2987*y['Output 6']
	 + 39.5518*y['Output 7'] <= 0;

subject to efficiency_constraint[2]:
	-91.5586*x['Input 1'] - 1.1948*x['Input 2'] - 77.4837*x['Input 3'] + 
	9.8605*y['Output 1'] + 70.3838*y['Output 2'] + 127.559*y['Output 3'] + 
	59.4214*y['Output 4'] + 88.7819*y['Output 5'] + 46.5052*y['Output 6']
	 + 2.3613*y['Output 7'] <= 0;

subject to efficiency_constraint[3]:
	-43.7093*x['Input 1'] - 0.8526*x['Input 2'] - 185.228*x['Input 3'] + 
	8.741*y['Output 1'] + 31.4517*y['Output 2'] + 2.281*y['Output 3'] + 
	31.1873*y['Output 4'] + 30.2933*y['Output 5'] + 18.

In [ ]:
ampl.setOption('solver', 'cbc')
ampl.setOption('solver_msg', '0')
count_efficient = 0  # Initialize a counter for efficient hospitals

for k in Z:
    ampl.param['k'] = k
    ampl.solve()
    obj = ampl.getObjective('total_value_objective')
    if obj.get().value() > 0.999:
        print("%s %f" % (k, obj.get().value()))
        count_efficient += 1

# Print the total number of efficient hospitals
print("Total count of efficient hospitals:", count_efficient)



cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: 9 1.000000
cbc 2.10.10: cbc 2.10.10: 11 1.000000
cbc 2.10.10: 12 1.000000
cbc 2.10.10: cbc 2.10.10: 14 1.000000
cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: cbc 2.10.10: 